In [1]:
import json
import torch
import numpy as np
import faiss

In [2]:
with open('sentences.json') as f:
    sentences = json.load(f)
    
sentences

['A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people.',
 'The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century.',
 'Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.',
 'As of 2018, approximately 37.9 million people are infected with HIV globally.',
 'Cholera is an infection of the small intestine by some strains of the bacterium Vibrio cholerae.',
 'Classic cholera symptom is large amounts of watery diarrhea that lasts a few days. Vomiting and muscle cramps may also occur. Diarrhea can be so severe that it leads within hours to severe dehydration and electrolyte imbalance.',
 'The COVID-19 pandemic, also known as the coronavirus pandemic, is an ongoing pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome 

In [3]:
from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained("distilbert-base-uncased")
#text = "Replace me by any text you'd like."
#encoded_input = tokenizer(text, return_tensors='pt')
#output = model(**encoded_input)[0].detach().squeeze()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')

In [5]:
encoded_input

{'input_ids': tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
vector = model(**encoded_input)[0].detach().squeeze()

In [7]:
vector.size()

torch.Size([12, 768])

In [8]:
def encode(document: str) -> torch.Tensor:
  tokens = tokenizer(document, return_tensors='pt')
  vector = model(**tokens)[0].detach().squeeze()
  return torch.mean(vector, dim=0)

In [9]:
averaged_vectors = [encode(sentence) for sentence in sentences ]

In [10]:

index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(
    np.array([t.numpy() for t in averaged_vectors]),
    # the IDs will be 0 to len(documents)
    np.array(range(0, len(sentences)))
)

In [11]:
index.ntotal

11

In [16]:
def search(query: str, k=1):
    encoded = encode(query)
    unsqueezed = torch.unsqueeze(encoded, 0)
    encoded_query = unsqueezed.numpy()
    top_k = index.search(encoded_query, k)
    scores = top_k[0][0]
    results = [sentences[_id] for _id in top_k[1][0]]
    return list(zip(results, scores))

In [24]:
with open('questions.json') as f:
    questions = json.load(f)
    
for question in questions:
    print("Question: ", question)
    answers = search(question, 5)
    for i, answer in enumerate(answers):
        print(f"Answer{i}: ",  answer)

Question:  How many people have died during Black Death?
Answer0:  ('As of 2018, approximately 37.9 million people are infected with HIV globally.', 52.613445)
Answer1:  ('A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people.', 51.774715)
Answer2:  ('The death toll of Spanish Flu is estimated to have been somewhere between 17 million and 50 million, and possibly as high as 100 million, making it one of the deadliest pandemics in human history.', 51.187973)
Answer3:  ('The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century.', 50.574467)
Answer4:  ('The Spanish flu, also known as the 1918 flu pandemic, was an unusually deadly influenza pandemic caused by the H1N1 influenza A virus.', 47.014305)
Question:  Which diseases can be transmitted by animals?
Answer0:  (